In [ ]:
!pip install google-cloud-speech
!pip install pyaudio

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pyaudio)


In [ ]:
from google.cloud import speech
import io

In [ ]:
import os

# Set the environment variable with your JSON key file name
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "agile-sanctum-442220-v0-546e65f56962.json"

In [ ]:
from google.cloud import speech

# Initialize the Speech-to-Text client
client = speech.SpeechClient()

print("Google Cloud Speech-to-Text API is ready!")

Google Cloud Speech-to-Text API is ready!


In [ ]:
# URI of the uploaded audio file in Google Cloud Storage
gcs_uri = "gs://audiofilesbuckethahaha/OSR_fr_000_0041_8k.wav"

# Configure the audio and recognition settings
audio = speech.RecognitionAudio(uri=gcs_uri)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=8000,  # Sample rate should match your audio file
    language_code="fr-FR",   # French language code
)

# Perform LongRunningRecognize
operation = client.long_running_recognize(config=config, audio=audio)

# Wait for the operation to complete
print("Processing the audio file, please wait...")
response = operation.result()

# Print the transcription results
for result in response.results:
    print("Transcript: {}".format(result.alternatives[0].transcript))

Processing the audio file, please wait...
Transcript: pourrais-je avoir un verre d'eau
Transcript:  mets SNCF assurera un train sur 3
Transcript:  les coupoles de l'immense palais s'écroulèrent
Transcript:  on apercevait la voile blanche du petit plateau
Transcript:  ils sont y ni douleur ni secousses
Transcript:  sois toujours à la mer tu gagneras
Transcript:  les langues vont bon train
Transcript:  le soleil joue à cache-cache avec les nuages
Transcript:  l'animal le regarde avec reconnaissance
Transcript:  ils sont juste en face sur le toit
Transcript:  parfois il me questionne sur ma vie
Transcript:  je garde un souvenir ému de nuit
Transcript:  la brebis est dans sa litière sèche
Transcript:  c'est véritablement le nerf de la guerre
Transcript:  d'habitude j'ai dit qu'il faut faire ça
Transcript:  nous nous sommes qui ont triste et très abattu
Transcript:  sa discrétion m'étonnait énormément
Transcript:  comprenez-vous ma joie


In [ ]:
!pip install google-cloud-translate

In [ ]:
from google.cloud import translate_v2 as translate

# Initialize the Translation client
translate_client = translate.Client()

# Translate the transcriptions from French to English
translated_texts = []

for result in response.results:
    original_text = result.alternatives[0].transcript
    translation = translate_client.translate(original_text, target_language='en')
    translated_texts.append(translation['translatedText'])

# Print the translated texts
for text in translated_texts:
    print("Translated: {}".format(text))

Translated: could i have a glass of water
Translated:  mets SNCF will provide one train in three
Translated:  the domes of the immense palace collapsed
Translated:  we could see the white sail of the small plateau
Translated:  they are there neither pain nor jolts
Translated:  always be at sea you will win
Translated:  languages are going well
Translated:  the sun plays hide and seek with the clouds
Translated:  the animal looks at him with recognition
Translated:  they are just across the street on the roof
Translated:  sometimes he asks me about my life
Translated:  I have a fond memory of the night
Translated:  the sheep is in its dry litter
Translated:  This is truly the crux of the matter
Translated:  usually i said that we have to do this
Translated:  we are who are sad and very dejected
Translated:  His discretion surprised me enormously.
Translated:  do you understand my joy


In [ ]:
!pip install google-cloud-texttospeech

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
!pip install pydub

In [ ]:
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
from pydub import AudioSegment
import os

def generate_speech_and_combine(translated_texts, language_code='en-US', gender=texttospeech.SsmlVoiceGender.FEMALE, output_filename='combined_output.mp3'):
    combined = AudioSegment.empty()  # Start with an empty audio segment

    for idx, text in enumerate(translated_texts):
        # Set the text input to be synthesized
        synthesis_input = texttospeech.SynthesisInput(text=text)

        # Build the voice request, select the language code ("en-US") and the voice gender
        voice = texttospeech.VoiceSelectionParams(
            language_code=language_code,
            ssml_gender=gender
        )

        # Specify the audio file configuration
        audio_config = texttospeech.AudioConfig(
            audio_encoding=texttospeech.AudioEncoding.MP3
        )

        # Perform the text-to-speech request
        response = client_tts.synthesize_speech(
            input=synthesis_input, voice=voice, audio_config=audio_config
        )

        # Save the audio content to a temporary file
        temp_filename = f"temp_{idx+1}.mp3"
        with open(temp_filename, "wb") as out:
            out.write(response.audio_content)

        # Load the temporary file and append to the combined audio
        temp_audio = AudioSegment.from_mp3(temp_filename)
        combined += temp_audio  # Append the audio

        # Remove the temporary file
        os.remove(temp_filename)

    # Export the combined audio to a single MP3 file
    combined.export(output_filename, format="mp3")
    print(f"All translations have been combined into {output_filename}")

# List of translated texts (from your previous result)
translated_texts = [
    "Could I have a glass of water",
    "mets SNCF will provide one train in three",
    "The domes of the immense palace collapsed",
    "We could see the white sail of the small plateau",
    "They are there neither pain nor jolts",
    "Always be at sea you will win",
    "Languages are going well",
    "The sun plays hide and seek with the clouds",
    "The animal looks at him with recognition",
    "They are just across the street on the roof",
    "Sometimes he asks me about my life",
    "I have a fond memory of the night",
    "The sheep is in its dry litter",
    "This is truly the crux of the matter",
    "Usually I said that we have to do this",
    "We are who are sad and very dejected",
    "His discretion surprised me enormously.",
    "Do you understand my joy"
]

# Generate speech and combine it into a single file
generate_speech_and_combine(translated_texts, language_code="en-US")

All translations have been combined into combined_output.mp3


In [ ]:
from google.cloud import speech, translate_v2 as translate, texttospeech

# Initialize clients
speech_client = speech.SpeechClient()
translate_client = translate.Client()
tts_client = texttospeech.TextToSpeechClient()

def transcribe_audio(audio_uri):
    # Speech-to-Text API logic
    audio = speech.RecognitionAudio(uri=audio_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=8000,  # Adjust based on your audio file
        language_code="fr-FR",   # French language
    )

    # Perform transcription
    operation = speech_client.long_running_recognize(config=config, audio=audio)
    response = operation.result()

    # Extract transcript
    transcripts = [result.alternatives[0].transcript for result in response.results]
    return transcripts

def translate_text(text, target_language="en"):
    # Translation logic
    translation = translate_client.translate(text, target_language=target_language)
    return translation['translatedText']

def text_to_speech(text, output_audio_file):
    # Text-to-Speech logic
    synthesis_input = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(language_code="en-US", ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL)
    audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)

    response = tts_client.synthesize_speech(input=synthesis_input, voice=voice, audio_config=audio_config)

    # Save the output audio
    with open(output_audio_file, "wb") as out:
        out.write(response.audio_content)

In [ ]:
!sudo apt-get install portaudio19-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 0s (666 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEn

In [ ]:
!pip install pyaudio

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=PyAudio-0.2.14-cp310-cp310-linux_x86_64.whl size=63862 sha256=58c486885cce0042adb90838872abc52391b042f32c34c70a31c69c3942a3344
  Stored in directory: /root/.cache/pip/wheels/d6/21/f4/0b51d41ba79e51b16295cbb096ec49f334792814d545b508c5
Successfully built pyaudio


In [ ]:
!pip install gtts

In [ ]:
import os
from google.cloud import speech, translate_v2 as translate
from gtts import gTTS
from pydub import AudioSegment
import time

# Set up Google Cloud credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'agile-sanctum-442220-v0-546e65f56962.json'

# Initialize clients for speech recognition and translation
speech_client = speech.SpeechClient()
translate_client = translate.Client()

# Set the GCS URI for the audio file (replace with your actual URI)
audio_uri = "gs://audiofilesbuckethahaha/OSR_in_000_0062_16k.wav"

# Audio configuration for speech recognition
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,  # Audio encoding type
    sample_rate_hertz=16000,  # The sample rate of the audio file
    language_code="hi-IN",  # Language code for Hindi
)

# Audio object with the GCS URI
audio = speech.RecognitionAudio(uri=audio_uri)

# Start speech-to-text processing
print("Starting audio processing...")

try:
    # Request LongRunning speech recognition from Google Cloud (for audio longer than 1 minute)
    operation = speech_client.long_running_recognize(config=config, audio=audio)

    print("Processing... Please wait.")

    # Wait for the operation to complete
    response = operation.result(timeout=90)  # Set a timeout value if needed

    # Initialize an empty AudioSegment list to hold the MP3 audio segments
    audio_segments = []

    # Process the recognition results and translate each sentence
    for result in response.results:
        transcript = result.alternatives[0].transcript
        print(f"Transcript: {transcript}")

        # Translate each sentence to English
        translation = translate_client.translate(transcript, target_language='en')
        translated_text = translation['translatedText']
        print(f"Translated Text: {translated_text}")

        # Convert the translated text to speech (MP3)
        tts = gTTS(translated_text, lang='en')
        tts.save("temp.mp3")

        # Load the MP3 file into pydub and append it to the audio_segments list
        sentence_audio = AudioSegment.from_mp3("temp.mp3")
        audio_segments.append(sentence_audio)

        # Add a brief pause (500ms) after each sentence
        silence = AudioSegment.silent(duration=500)  # 500 milliseconds of silence
        audio_segments.append(silence)

    # Concatenate all audio segments (including pauses)
    final_audio = AudioSegment.empty()  # Start with an empty AudioSegment
    for segment in audio_segments:
        final_audio += segment  # Concatenate each segment

    # Export the final combined audio to an MP3 file
    final_audio.export("translated_audio_with_pauses.mp3", format="mp3")
    print("Audio file with pauses saved as 'translated_audio_with_pauses.mp3'.")

except Exception as e:
    print(f"Error occurred: {e}")

Starting audio processing...
Processing... Please wait.
Transcript: केशव के घर में चार खिड़कियां हैं कई लोग कुमार को पसंद करते हैं
Translated Text: Keshav&#39;s house has four windows Many people like Kumar
Transcript:  तुम्हारे खरगोश का रंग सफेद है
Translated Text:  your rabbit is white
Transcript:  आपकी गायक कल से यहां है
Translated Text:  your singer is here since yesterday
Transcript:  कल का खाना सुलेखा की डालकर बनाएगी
Translated Text:  Tomorrow she will cook food by adding Sulekha&#39;s
Transcript:  अक्षय की खीर गरम हो गई
Translated Text:  Akshay&#39;s kheer got hot
Transcript:  मैंने कल ख्वाब में एक खूबसूरत गाड़ी देखी
Translated Text:  I saw a beautiful car in my dream yesterday.
Transcript:  यह खबर सुनकर रघु खुशी से पागल हो गया
Translated Text:  Raghu became crazy with happiness on hearing this news
Transcript:  कूट की लड़ाई में कई जवान घायल हुए
Translated Text:  Many soldiers were injured in the battle of Koot
Transcript:  कैकई के आगे गोरी चल रही है
Translated Text:  Gori is wa